In [5]:
import os
# Set this before importing PyTorch to prevent memory fragmentation, as per expert advice.
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [1]:
# --- Imports and Configuration ---
import os
import sys
import glob
import pandas as pd
import numpy as np
import cv2
import torch
import timm
import pydicom
import SimpleITK as sitk
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

# --- Configuration ---
DETECTOR_WEIGHTS = 'yolov5_runs/train/baseline_fold0/weights/best.pt'
CLASSIFIER_WEIGHTS = 'classifier_fold0_best.pth'
YOLO_REPO_DIR = 'yolov5'

TEST_DIR = 'test/'
TEST_PNG_DIR = 'test_png/'
SAMPLE_SUBMISSION = 'sample_submission.csv'

IMG_SIZE_DET = 640  # Image size for detector
IMG_SIZE_CLS = 512  # Image size for classifier
BATCH_SIZE = 16
NUM_WORKERS = 4

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {DEVICE}")
print(f"Detector weights: {DETECTOR_WEIGHTS}")
print(f"Classifier weights: {CLASSIFIER_WEIGHTS}")

Using device: cuda
Detector weights: yolov5_runs/train/baseline_fold0/weights/best.pt
Classifier weights: classifier_fold0_best.pth


In [2]:
# --- Preprocess Test DICOMs ---
print("--- Preprocessing Test DICOMs to PNG --- ")
os.makedirs(TEST_PNG_DIR, exist_ok=True)

df_sub = pd.read_csv(SAMPLE_SUBMISSION)
# The sample submission has two rows per study for the two prediction types.
# We only need one row per image to find the files.
df_sub['image_id'] = df_sub['id'].apply(lambda x: x.split('_')[0])
df_test_imgs = df_sub[df_sub.id.str.contains('_image')].copy()

# Find all test dcm files
test_dcm_files = glob.glob(f'{TEST_DIR}/*/*/*.dcm')
image_id_to_path = {os.path.basename(p).replace('.dcm', ''): p for p in test_dcm_files}
df_test_imgs['dcm_path'] = df_test_imgs['image_id'].map(image_id_to_path)

def process_dicom_with_sitk(row, output_dir):
    image_id = row['image_id']
    dcm_path = row['dcm_path']
    if pd.isna(dcm_path): return
    save_path = os.path.join(output_dir, f"{image_id}.png")
    if os.path.exists(save_path): return

    try:
        dicom_meta = pydicom.dcmread(dcm_path, stop_before_pixels=True)
        sitk_image = sitk.ReadImage(dcm_path)
        data = sitk.GetArrayFromImage(sitk_image).squeeze()
        if 'RescaleSlope' in dicom_meta and 'RescaleIntercept' in dicom_meta:
            slope = float(dicom_meta.RescaleSlope)
            intercept = float(dicom_meta.RescaleIntercept)
            data = data * slope + intercept
        if dicom_meta.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
        center, width = -600, 1500
        min_val, max_val = center - width // 2, center + width // 2
        data = np.clip(data, min_val, max_val)
        data = data - np.min(data)
        data = data / (np.max(data) + 1e-6)
        data = (data * 255).astype(np.uint8)
        cv2.imwrite(save_path, data)
    except Exception as e:
        print(f"Error processing {dcm_path}: {e}")

for _, row in tqdm(df_test_imgs.iterrows(), total=len(df_test_imgs), desc="Processing Test DICOMs"):
    process_dicom_with_sitk(row, TEST_PNG_DIR)

print(f"Finished processing test images. Check '{TEST_PNG_DIR}'.")

--- Preprocessing Test DICOMs to PNG --- 


Processing Test DICOMs:   0%|          | 0/638 [00:00<?, ?it/s]

Processing Test DICOMs:   0%|          | 1/638 [00:00<01:06,  9.64it/s]

Processing Test DICOMs:   0%|          | 2/638 [00:00<01:23,  7.60it/s]

Processing Test DICOMs:   1%|          | 4/638 [00:00<00:56, 11.20it/s]

...

Processing Test DICOMs:  99%|█████████▉| 633/638 [01:26<00:00,  8.97it/s]

Processing Test DICOMs:  99%|█████████▉| 634/638 [01:27<00:00,  8.47it/s]

Processing Test DICOMs: 100%|█████████▉| 636/638 [01:27<00:00, 10.14it/s]

Processing Test DICOMs: 100%|██████████| 638/638 [01:27<00:00,  7.31it/s]

Finished processing test images. Check 'test_png/'.


In [3]:
# --- Get Test Image Dimensions ---
print("Reading DICOM metadata to get test image dimensions...")
df_test_imgs['img_height'] = 0
df_test_imgs['img_width'] = 0

for index, row in tqdm(df_test_imgs.iterrows(), total=len(df_test_imgs), desc="Reading Test DICOM metadata"):
    if pd.isna(row['dcm_path']): continue
    try:
        dicom_meta = pydicom.dcmread(row['dcm_path'], stop_before_pixels=True)
        df_test_imgs.loc[index, 'img_height'] = dicom_meta.Rows
        df_test_imgs.loc[index, 'img_width'] = dicom_meta.Columns
    except Exception as e:
        print(f"Could not read dimensions for {row['image_id']}: {e}")

print("Finished getting test image dimensions.")
df_test_imgs.head()

Reading DICOM metadata to get test image dimensions...


Reading Test DICOM metadata:   0%|          | 0/638 [00:00<?, ?it/s]

Reading Test DICOM metadata:  30%|██▉       | 190/638 [00:00<00:00, 1892.15it/s]

Reading Test DICOM metadata:  60%|██████    | 383/638 [00:00<00:00, 1913.66it/s]

Reading Test DICOM metadata:  90%|█████████ | 577/638 [00:00<00:00, 1925.57it/s]

Reading Test DICOM metadata: 100%|██████████| 638/638 [00:00<00:00, 1917.61it/s]

Finished getting test image dimensions.


,id,PredictionString,image_id,dcm_path,img_height,img_width
606,004cbd797cd1_image,none 1 0 0 1 1,004cbd797cd1,test/30e45593ba08/c5c0a57e6e4c/004cbd797cd1.dcm,2991,2992
607,008ca392cff3_image,none 1 0 0 1 1,008ca392cff3,test/39a80a14bfda/088a20979e31/008ca392cff3.dcm,3480,4240
608,00b8180bd3a8_image,none 1 0 0 1 1,00b8180bd3a8,test/dadc2e3842e5/e7866b8dd3df/00b8180bd3a8.dcm,2991,2992
609,00e3a7e91a34_image,none 1 0 0 1 1,00e3a7e91a34,test/74ba8f2badcb/c8755f476425/00e3a7e91a34.dcm,1760,2140
610,0124f624dacb_image,none 1 0 0 1 1,0124f624dacb,test/0acf45b01bdf/11b150218e91/0124f624dacb.dcm,3480,4240


In [7]:
import subprocess
# Check GPU memory after restarting other kernels using a robust method
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
    print(result.stdout)
except (subprocess.CalledProcessError, FileNotFoundError) as e:
    print(f"Could not run nvidia-smi: {e}")

Fri Sep 26 01:05:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |    4352MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
# --- Run YOLOv5 Detector (Batched, FP16) ---
print("--- Running YOLOv5 Detector on Test Images ---")

if YOLO_REPO_DIR not in sys.path:
    sys.path.append(YOLO_REPO_DIR)

model_det = torch.hub.load(
    YOLO_REPO_DIR, 'custom', path=DETECTOR_WEIGHTS, source='local', force_reload=True
).to(DEVICE).eval()

# Set confidence threshold on the model object itself, not in the forward call
model_det.conf = 0.001

# FP16 to cut memory, as per expert advice
if DEVICE.type == 'cuda':
    model_det.half()

test_png_files = sorted(glob.glob(f'{TEST_PNG_DIR}/*.png'))
print(f"Found {len(test_png_files)} test PNGs to process.")

DET_BATCH = 16  # As suggested by expert; can be tuned
predictions = []

with torch.inference_mode():
    for i in tqdm(range(0, len(test_png_files), DET_BATCH), desc="YOLOv5 Inference"):
        batch_files = test_png_files[i:i+DET_BATCH]
        # The 'conf' argument is set on the model, not passed here.
        results = model_det(batch_files, size=IMG_SIZE_DET)
        # The expert noted AutoShape returns coordinates in original image space, so no manual scaling needed.
        for img_path, df in zip(batch_files, results.pandas().xyxy):
            image_id = os.path.basename(img_path).replace('.png', '')
            if len(df):
                df = df[['xmin','ymin','xmax','ymax','confidence','class','name']].copy()
                df['image_id'] = image_id
                predictions.append(df)
        del results
        if DEVICE.type == 'cuda':
            torch.cuda.empty_cache()

if predictions:
    df_det_preds = pd.concat(predictions, ignore_index=True)
    print(f"Generated {len(df_det_preds)} bounding box predictions.")
    display(df_det_preds.head())
else:
    print("No bounding boxes were predicted.")
    df_det_preds = pd.DataFrame()

YOLOv5 🚀 v7.0-432-g725b922e Python-3.11.0rc1 torch-2.8.0+cu128 CUDA:0 (NVIDIA A10-24Q, 24291MiB)



Fusing layers... 


Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


Adding AutoShape... 


--- Running YOLOv5 Detector on Test Images ---
Found 638 test PNGs to process.


...

/var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
YOLOv5 Inference:  98%|█████████▊| 39/40 [00:27<00:00,  1.51it/s]

/var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
YOLOv5 Inference: 100%|██████████| 40/40 [00:27<00:00,  1.62it/s]

YOLOv5 Inference: 100%|██████████| 40/40 [00:27<00:00,  1.44it/s]

Generated 47674 bounding box predictions.


,xmin,ymin,xmax,ymax,confidence,class,name,image_id
0,1844.287598,1003.456299,2409.962646,2141.818848,0.041931,0,opacity,004cbd797cd1
1,1734.425049,1376.287476,2342.175049,2353.362549,0.036346,0,opacity,004cbd797cd1
2,673.200012,1154.225098,1317.181274,2294.925049,0.028549,0,opacity,004cbd797cd1
3,638.721924,1760.806274,1323.025024,2524.000244,0.013275,0,opacity,004cbd797cd1
4,1801.043823,1708.212524,2463.725098,2570.750244,0.012672,0,opacity,004cbd797cd1


In [10]:
# --- Classifier Inference ---
print("--- Running Classifier on Test Images ---")

# --- Dataset and Transforms (similar to training, but for inference) ---
class SIIMTestClassifierDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.image_paths = df['image_path'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image

def get_cls_test_transforms(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

# --- Prepare DataLoader ---
df_test_imgs['image_path'] = df_test_imgs['image_id'].apply(lambda x: os.path.join(TEST_PNG_DIR, f"{x}.png"))
test_cls_dataset = SIIMTestClassifierDataset(df_test_imgs, transform=get_cls_test_transforms(IMG_SIZE_CLS))
test_cls_loader = DataLoader(test_cls_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# --- Load Model ---
MODEL_NAME = 'tf_efficientnet_b4_ns'
NUM_CLASSES = 4
model_cls = timm.create_model(MODEL_NAME, pretrained=False, num_classes=NUM_CLASSES)
model_cls.load_state_dict(torch.load(CLASSIFIER_WEIGHTS, map_location=DEVICE))
model_cls.to(DEVICE)
model_cls.eval()

# --- Run Inference ---
all_cls_preds = []
with torch.no_grad():
    for images in tqdm(test_cls_loader, desc="Classifier Inference"):
        images = images.to(DEVICE)
        with torch.cuda.amp.autocast():
            outputs = model_cls(images)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
        all_cls_preds.append(probs)

all_cls_preds = np.concatenate(all_cls_preds)

# --- Store Predictions in DataFrame ---
label_cols = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
df_cls_preds = pd.DataFrame(all_cls_preds, columns=label_cols)
df_cls_preds['image_id'] = df_test_imgs['image_id'].values

print(f"Generated {len(df_cls_preds)} classifier predictions.")
display(df_cls_preds.head())

--- Running Classifier on Test Images ---


Classifier Inference:   0%|          | 0/40 [00:00<?, ?it/s]

/tmp/ipykernel_6896/2339599941.py:50: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Classifier Inference:   2%|▎         | 1/40 [00:05<03:17,  5.05s/it]

Classifier Inference:   8%|▊         | 3/40 [00:05<00:50,  1.35s/it]

...

Classifier Inference:  98%|█████████▊| 39/40 [00:13<00:00,  3.28it/s]

Classifier Inference: 100%|██████████| 40/40 [00:17<00:00,  1.13it/s]

Classifier Inference: 100%|██████████| 40/40 [00:17<00:00,  2.30it/s]

Generated 638 classifier predictions.


,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,image_id
0,0.171116,0.336176,0.369578,0.123130,004cbd797cd1
1,0.032415,0.811077,0.133443,0.023065,008ca392cff3
2,0.191343,0.455548,0.219855,0.133253,00b8180bd3a8
3,0.292518,0.493116,0.129425,0.084941,00e3a7e91a34
4,0.130996,0.677049,0.122520,0.069436,0124f624dacb


In [16]:
# --- Format and Combine Predictions for Submission ---
print("--- Formatting predictions into submission format ---")

# --- 1. Format Image-level (Detection) Predictions ---
image_preds = {}
if not df_det_preds.empty:
    # Recommended robustness: clamp, cast, and ensure w/h >= 1
    df_det_preds['xmin'] = df_det_preds['xmin'].clip(lower=0).round().astype(int)
    df_det_preds['ymin'] = df_det_preds['ymin'].clip(lower=0).round().astype(int)
    df_det_preds['xmax'] = df_det_preds['xmax'].clip(lower=0).round().astype(int)
    df_det_preds['ymax'] = df_det_preds['ymax'].clip(lower=0).round().astype(int)
    df_det_preds['w'] = (df_det_preds['xmax'] - df_det_preds['xmin']).clip(lower=1).astype(int)
    df_det_preds['h'] = (df_det_preds['ymax'] - df_det_preds['ymin']).clip(lower=1).astype(int)

    for image_id, group in df_det_preds.groupby('image_id'):
        pred_strings = ' '.join(
            f"opacity {row.confidence:.4f} {row.xmin} {row.ymin} {row.w} {row.h}"
            for row in group.itertuples(index=False)
        )
        image_preds[image_id] = pred_strings

# Create the final image-level submission dataframe
image_sub_list = []
for image_id in df_test_imgs['image_id'].unique():
    pred_string = image_preds.get(image_id, 'none 1 0 0 1 1')
    image_sub_list.append({'id': f"{image_id}_image", 'PredictionString': pred_string})
df_image_sub = pd.DataFrame(image_sub_list)

print(f"Formatted {len(df_image_sub)} image-level predictions.")
display(df_image_sub.head())

# --- 2. Format Study-level (Classification) Predictions ---
# Get StudyInstanceUID for each image
df_test_imgs['StudyInstanceUID'] = df_test_imgs['dcm_path'].apply(lambda x: x.split('/')[-3] if pd.notna(x) else None)
df_cls_preds_with_study = pd.merge(df_cls_preds, df_test_imgs[['image_id', 'StudyInstanceUID']], on='image_id', how='left')

# Aggregate image-level predictions to study-level by taking the mean
label_cols = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
df_study_preds = df_cls_preds_with_study.groupby('StudyInstanceUID')[label_cols].mean().reset_index()

# Format the prediction string based on direct inspection of sample_submission.csv
def format_study_string_from_sample(row):
    # Find the class with the highest probability
    max_prob_col = row[label_cols].idxmax()
    
    # Map the column name to the lowercase token
    token_map = {
        'Negative for Pneumonia': 'negative',
        'Typical Appearance': 'typical',
        'Indeterminate Appearance': 'indeterminate',
        'Atypical Appearance': 'atypical'
    }
    class_token = token_map[max_prob_col]
    
    # Format the string like the sample file: 'classname 1 0 0 1 1'
    return f"{class_token} 1 0 0 1 1"

df_study_preds['PredictionString'] = df_study_preds.apply(format_study_string_from_sample, axis=1)
df_study_preds['id'] = df_study_preds['StudyInstanceUID'].apply(lambda x: f"{x}_study")
df_study_sub = df_study_preds[['id', 'PredictionString']]

print(f"\nFormatted {len(df_study_sub)} study-level predictions.")
display(df_study_sub.head())

# --- 3. Combine and Save Final Submission ---
submission_df = pd.concat([df_image_sub, df_study_sub], ignore_index=True)

# Sort the submission file by ID to match the sample submission order.
submission_df = submission_df.sort_values(by='id').reset_index(drop=True)

submission_df.to_csv('submission.csv', index=False)

print("\nFinal submission file created: submission.csv")
display(submission_df.head())
print(f"Total rows in submission: {len(submission_df)}")

--- Formatting predictions into submission format ---


Formatted 638 image-level predictions.


,id,PredictionString
0,004cbd797cd1_image,opacity 0.0419 1844 1003 566 1139 opacity 0.03...
1,008ca392cff3_image,opacity 0.3464 2329 665 940 1209 opacity 0.147...
2,00b8180bd3a8_image,opacity 0.0605 8 1956 677 1035 opacity 0.0317 ...
3,00e3a7e91a34_image,opacity 0.0417 0 287 333 893 opacity 0.0333 14...
4,0124f624dacb_image,opacity 0.0614 725 1065 894 1556 opacity 0.058...



Formatted 606 study-level predictions.


,id,PredictionString
0,000c9c05fd14_study,typical 1 0 0 1 1
1,00c74279c5b7_study,typical 1 0 0 1 1
2,00ccd633fb0e_study,typical 1 0 0 1 1
3,00e936c58da6_study,typical 1 0 0 1 1
4,01206a422293_study,typical 1 0 0 1 1



Final submission file created: submission.csv


,id,PredictionString
0,000c9c05fd14_study,typical 1 0 0 1 1
1,004cbd797cd1_image,opacity 0.0419 1844 1003 566 1139 opacity 0.03...
2,008ca392cff3_image,opacity 0.3464 2329 665 940 1209 opacity 0.147...
3,00b8180bd3a8_image,opacity 0.0605 8 1956 677 1035 opacity 0.0317 ...
4,00c74279c5b7_study,typical 1 0 0 1 1


Total rows in submission: 1244


In [15]:
# --- Inspect Sample Submission Format ---
print("--- Inspecting sample_submission.csv format ---")
df_sample = pd.read_csv(SAMPLE_SUBMISSION)

sample_study_string = df_sample[df_sample.id.str.contains('_study')]['PredictionString'].iloc[0]
print("Sample study-level prediction string:")
print(f"'{sample_study_string}'")
print(f"String ends with space: {sample_study_string.endswith(' ')}")

sample_image_string = df_sample[df_sample.id.str.contains('_image')]['PredictionString'].iloc[0]
print("\nSample image-level prediction string (no detection):")
print(f"'{sample_image_string}'")

--- Inspecting sample_submission.csv format ---
Sample study-level prediction string:
'negative 1 0 0 1 1'
String ends with space: False

Sample image-level prediction string (no detection):
'none 1 0 0 1 1'
